# Carrega o arquivo csv e cria o dataframe

In [1]:
import pandas as pd
import seaborn as sns

df = pd.read_csv(r"C:\Users\rodol\Videos\Desktop\Projetos\Churn_funcionarios\Base_RH_csv2.csv", sep=';', encoding='latin-1')

In [2]:
display(df)

,ID,Funcionário_deixou_a_empresa,Idade,Frequência de Viagens,Distância_do_trabalho,Formação,E-Sat,Gênero,Estado_Civil,Salário,...,Faz_hora_extras?,Perc_de_aumento,Qte_ações_da_empresa,Tempo_de_carreira,Horas_de_treinamento,Equilibrio_de_Vida,Tempo_de_empresa,Anos_no_mesmo_cargo,Anos_desde_a_ultima_promocao,Anos_com_o_mesmo_chefe
0,1,1,41,Viaja raramente,1,Ensino Técnico,Médio,M,Solteiro,5993,...,Sim,11,0,8,0,Ruim,6,4,0,5
1,2,0,49,Viaja frequentemente,8,Ensino Médio,Alto,H,Casado,5130,...,Não,23,1,10,3,Muito bom,10,7,1,7
2,3,1,37,Viaja raramente,2,Ensino Técnico,Muito Alto,H,Solteiro,2090,...,Sim,15,0,7,3,Muito bom,0,0,0,0
3,4,0,33,Viaja frequentemente,3,Mestrado,Muito Alto,M,Casado,2909,...,Sim,11,0,8,3,Muito bom,8,7,3,0
4,5,0,27,Viaja raramente,2,Ensino Médio,Baixo,H,Casado,3468,...,Não,12,1,6,3,Muito bom,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1466,0,36,Viaja frequentemente,23,Ensino Técnico,Alto,H,Casado,2571,...,Não,17,1,17,3,Muito bom,5,2,0,3
1466,1467,0,39,Viaja raramente,6,Ensino Médio,Muito Alto,H,Casado,9991,...,Não,15,1,9,5,Muito bom,7,7,1,7
1467,1468,0,27,Viaja raramente,4,Ensino Superior,Médio,H,Casado,6142,...,Sim,20,1,6,0,Muito bom,6,2,0,3
1468,1469,0,49,Viaja frequentemente,2,Ensino Superior,Muito Alto,H,Casado,5390,...,Não,14,0,17,3,Bom,9,6,0,8


In [5]:
import numpy as np
import pandas as pd

def iv_woe(data, target, bins=5, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns

    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target, 'ID'])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        
        # Calculate the number of events in each group (bin)
        d = d0.groupby("x", as_index=False, observed=True).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        
        # Calculate % of events in each group.
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

        # Calculate the non events in each group.
        d['Non-Events'] = d['N'] - d['Events']
        # Calculate % of non events in each group.
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

        # Calculate WOE by taking natural log of division of % of non-events and % of events
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        newDF.sort_values(by='IV', ascending=False, inplace=True, kind='quicksort')
        #woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            display(d)
    return display(newDF), #woeDF

In [7]:
iv_woe(data = df, target='Funcionário_deixou_a_empresa')

,Variable,IV
0,Faz_hora_extras?,0.400183
0,Qte_ações_da_empresa,0.319083
0,Salário,0.315757
0,Tempo_de_empresa,0.300408
0,Tempo_de_carreira,0.276188
0,Anos_no_mesmo_cargo,0.262800
0,Idade,0.258060
0,Anos_com_o_mesmo_chefe,0.255291
0,Estado_Civil,0.218898
0,Qte_Empresas_Trabalhadas,0.130967


(None,)